In [2]:
# Majority of this cell is copied from train_llm4syn.py by Okabe et al. to reproduce what they did

import os
from os.path import join
import numpy as np
import torch
import random
import math
# #Not importing a few of the packages, this code is not designed for training
# from sklearn.model_selection import KFold
# from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, set_seed
# from huggingface_hub import login
# import wandb
# from env_config import hf_api_key_w, data_path, hf_usn, wandb_project, seedn
seedn=42
data_path=os.environ['DATA_PATH']
from utils.data_config import separator_dict, gpt_model_dict, arrow_l2r
from utils.data import load_and_sample_data
# from utils.model_utils import setup_tokenizer, tokenize_dataset, get_epoch_lists
from utils.utilities import make_dict
random.seed(seedn)
task='tgt2ceq'
sample_ratio = 1
arrow = arrow_l2r 
separator, cut = separator_dict[task], ';'
dataset = load_and_sample_data(data_path, task, separator, te_ratio=0.1, cut=cut, arrow=arrow, sample_ratio=sample_ratio, save_idx_name=None)

from pprint import pprint
from collections import Counter

In [4]:
for rxnoi in ['CaTi4Cu3O12 ||',
              'Zr0.54Ti0.46PbO3 ||',
              'TiPbO3 ||',
              'Eu0.02La0.98AlO3 ||',
              'BaZrxTi1-xO3 ||',
              ]:
    print(rxnoi)
    for split in ['test','train']:
        filtered_dataset = dataset[split].filter(lambda example: example['label'] == rxnoi)
        if split=='test' and len(filtered_dataset)==0:
            print("\n\nNo test data!!!!\n\n")#, rxnoi, "in", task, "task")
            break
        print(split, "set, size of", len(filtered_dataset))
        pprint(Counter([example['text'] for example in filtered_dataset]).most_common())
        print()
    print("\n##############\n\n")

CaTi4Cu3O12 ||
test set, size of 11
[('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 CaCO3 -> 1 CaCu3Ti4O12 + 1 CO2', 10),
 ('CaTi4Cu3O12 || 4 TiO2 + 3 CuO + 1 Ca(OH)2 -> 1 CaCu3Ti4O12 + 1 H2O', 1)]

train set, size of 77
[('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 CaCO3 -> 1 CaCu3Ti4O12 + 1 CO2', 54),
 ('CaTi4Cu3O12 || 3 Cu(NO3)2·3H2O + 4 TiO2 + 1 Ca(NO3)2·4H2O -> 1 CaCu3Ti4O12 '
  '+ 13 H2O + 8 NO2 + 2 O2',
  3),
 ('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 Ca(OH)2 -> 1 CaCu3Ti4O12 + 1 H2O', 2),
 ('CaTi4Cu3O12 || 1 CaO + 4 TiO2 + 3 CuO -> 1 CaCu3Ti4O12', 2),
 ('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 CaCO3 -> 1 CaCu3Ti4.0O12 + 1 CO2', 1),
 ('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 CaCO3 -> 1 CaCu3Ti4O12 + 1 CO2 ', 1),
 ('CaTi4Cu3O12 || 3 CuO + 1 CaCO3 + 4 anatase-TiO2 -> 1 CaCu3Ti4O12 + 1 CO2',
  1),
 ('CaTi4Cu3O12 || 3 CuO() + 1 CaCO3 + 4 TiO2() -> 1 CaCu3Ti4O12 + 1 CO2', 1),
 ('CaTi4Cu3O12 || 3 CuO + 1 CaCO3 + 4 TiO2 -> 1 CaCu3Ti4O12 + 1 CO2', 1),
 ('CaTi4Cu3O12 || 1 CaO + 3 CuO + 4 TiO2 -> 1 CaCu3Ti4O12', 1),
 ('CaTi4C

In [6]:

for task in ['lhs2rhs', 'rhs2lhs', 'lhsope2rhs', 'rhsope2lhs', 'tgt2ceq', 'tgtope2ceq']:
    print(task)
    separator = separator_dict[task]
    dataset = load_and_sample_data(data_path, task, separator, te_ratio=0.1, cut=cut, arrow=arrow, sample_ratio=sample_ratio, save_idx_name=None)

    all_tgts=set([example['label'] for example in dataset['train']]+[example['label'] for example in dataset['test']])
    test_tgts=set([example['label'] for example in dataset['test']])
    count_notest, count_yestestnotrain, count_dataleaked = 0, 0, 0
    # First, create dictionaries to store label counts for each split
    train_labels = defaultdict(int)
    test_labels = defaultdict(int)

    # Count occurrences of each label in one pass
    for split in ['train', 'test']:
        for example in dataset[split]:
            if split == 'train':
                train_labels[example['label']] += 1
            else:
                test_labels[example['label']] += 1

    # Initialize counters
    count_notest = 0
    count_yestestnotrain = []
    count_dataleaked = 0

    # Process all unique labels
    for tgtoi in tqdm(test_tgts):
        if test_labels.get(tgtoi, 0) == 0:
            count_notest += 1
        elif train_labels.get(tgtoi, 0) == 0:
            count_yestestnotrain.append(test_labels.get(tgtoi, 0))
        else:
            count_dataleaked += 1
    
    print(f"Out of {len(test_tgts)} points in test data,\n{count_dataleaked}({round(count_dataleaked/len(test_tgts),3)}) with data leakage (in both test and train);\n{len(count_yestestnotrain)}({round(len(count_yestestnotrain)/len(test_tgts),3)}) without data leakage (in only test but not train)\n##############\n\n")#)")#,\n\t{count_notest} not in test data")

lhs2rhs


100%|██████████| 1797/1797 [00:00<00:00, 560300.65it/s]


Out of 1797 points in test data,
459(0.255) with data leakage (in both test and train);
1338(0.745) without data leakage (in only test but not train)
##############


rhs2lhs


100%|██████████| 1784/1784 [00:00<00:00, 885414.55it/s]


Out of 1784 points in test data,
584(0.327) with data leakage (in both test and train);
1200(0.673) without data leakage (in only test but not train)
##############


lhsope2rhs


...

Out of 1930 points in test data,
208(0.108) with data leakage (in both test and train);
1722(0.892) without data leakage (in only test but not train)
##############


tgt2ceq


100%|██████████| 1715/1715 [00:00<00:00, 1165084.44it/s]


Out of 1715 points in test data,
530(0.309) with data leakage (in both test and train);
1185(0.691) without data leakage (in only test but not train)
##############


tgtope2ceq


100%|██████████| 1940/1940 [00:00<00:00, 908648.77it/s]

Out of 1940 points in test data,
130(0.067) with data leakage (in both test and train);
1810(0.933) without data leakage (in only test but not train)
##############


